<p>
<img src="../imgs/EII-ULPGC-logo.jpeg" width="430px" align="right">

# **NOTEBOOK 7.1**
---

# **Visual Transformers**


## **¿Qué es un Visual Transformer?**

Un Visual Transformer (a veces llamado Vision Transformer o ViT) es un modelo de deep learning que aplica la arquitectura Transformer, que originalmente revolucionó el procesamiento de lenguaje natural, a visión por computador.

En vez de usar convoluciones (como en las CNN clásicas tipo ResNet), un Visual Transformer trata una imagen como si fuera una “secuencia de tokens”, igual que una frase. Cada token no es una palabra, sino un trocito de la imagen.

Objetivo típico: clasificación de imágenes, detección de objetos, segmentación, etc.


## **¿Por qué esto fue un cambio importante?**

Antes de los Transformers visuales, prácticamente todo en visión profunda eran CNNs. Las CNNs tienen *inductive biases* fuertes:

- Localidad: miran primero ventanas pequeñas.
- Equivarianza traslacional: detectar un borde arriba o abajo es parecido.

Eso es bueno… pero también limita:

- Cuesta modelar dependencias globales (relaciones entre partes lejanas de la imagen).
- Escalar a imágenes enormes no siempre es eficiente.

Los **Visual Transformers** hacen algo diferente:

- Usan atención global. Cualquier parte de la imagen puede “mirar” cualquier otra parte desde el primer bloque.
- Escalan muy bien con más datos y más tamaño de red.

Resultado: cuando entrenas con datasets gigantes (no sólo ImageNet, sino cientos de millones de imágenes), los Vision Transformers igualan o superan a las CNNs en muchas tareas.


## **¿Cómo funciona un Vision Transformer paso a paso?**

Vamos con el ViT básico [Dosovitskiy et al., 2020](https://arxiv.org/abs/2010.11929), que es el diseño canónico:

<div align="center">
<img src="./imgs/Vit.png" alt="ViT Architecture" width="600"/>
</div>

### **Patch embedding**

1. Tomas una imagen, por ejemplo 224×224×3.
2. La divides en parches no solapados, por ejemplo 16×16.

   ¿Cuántos parches salen?

     - En horizontal: 224 / 16 = 14
     - En vertical: 224 / 16 = 14
     - Total: 14 × 14 = 196 parches.
3. Cada parche de 16×16×3 píxeles se aplana (es un vector) y pasa por una proyección lineal → queda como un embedding de dimensión fija (por ejemplo 768).

Traducción: conviertes la imagen en una secuencia de 196 “tokens visuales”.

Esto es análogo a: “cada palabra → embedding”, pero aquí “cada parche → embedding”.

### **Token [CLS]**

Se añade un token especial de clase (`[CLS]`) al principio de la secuencia. Este token actuará como resumen global de toda la imagen. Al final lo pasas a una softmax para predecir la clase.

Igual que en BERT para texto.

### **Positional embeddings**

Los Transformers puros no saben de posiciones espaciales. Para que el modelo sepa dónde está cada parche dentro de la imagen, se suma a cada token un embedding posicional (aprendido o sinusoidal).

En visión esto es crucial, porque “ojo arriba + pico amarillo abajo” no es lo mismo que al revés (pájaro vs. sol en la playa).

### **Encoder Transformer**

Después, la secuencia pasa por varios bloques Transformer idénticos. Cada bloque tiene:

- Multi-Head Self-Attention (MHSA)
- MLP feed-forward grande
- Normalizaciones + conexiones residuales

#### **¿Qué hace la self-attention aquí?**

Calcula, para cada parche, con qué otros parches debe interactuar y cuánto. Matemáticamente, cada token genera Query, Key, Value. Con eso calcula pesos de atención y mezcla información.

Intuición: El parche que contiene, por ejemplo, “rueda de coche” puede prestar atención a “parche con faro delantero”, aunque estén lejos en la imagen. Eso le da contexto global muy pronto.

### **Clasificación**

Al final, tomamos el embedding final del token `[CLS]` y lo pasamos por una capa lineal → logits de clases.


## **Ventajas frente a CNNs**

1. **Contexto global desde el inicio**
   Las CNNs necesitan muchas capas y pooling para juntar información distante. En ViT, atención global lo hace en el primer bloque.

2. **Escalabilidad limpia**
   Aumentar el tamaño de un Transformer (más capas, más anchura, más cabezas) es bastante sistemático y ya está súper estudiado en NLP. Básicamente puedes “fabricar” modelos enormes y entrenarlos con datasets masivos.

3. **Unifica visión y lenguaje**
   Como el backbone es Transformer, enganchar texto y visión en el mismo espacio (por ejemplo CLIP: imagen ↔ texto) es natural. Esto disparó los modelos multimodales.


## **Inconvenientes / retos**

1. **Datos, datos, datos**
   Un ViT puro entrenado desde cero en un dataset relativamente pequeño (por ejemplo sólo ImageNet-1k con 1.2M imágenes) tiende a generalizar peor que una buena CNN inicial. Necesitan o:

   * preentrenamiento masivo en datasets enormes,
   * o técnicas de regularización/agumentación fuertes (como DeiT).

2. **Costo de atención cuadrático**
   Atención clásica tiene coste O(N²) en número de tokens.
   Si partes una imagen en muchos parches pequeños (por ejemplo, 14×14 = 196 tokens está bien... pero 32×32 = 1024 tokens ya duele), la memoria y el cómputo suben rápido.


In [ ]:
import torch
from transformers import pipeline

pipeline = pipeline(
    task="image-classification",
    model="google/vit-base-patch16-224",
    dtype=torch.float16,
    device=0
)
pipeline("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg")